In [1]:
import pandas as pd
import numpy as np

#read data from csv
data = pd.read_csv("data/training_data_utf8.csv")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#define function to transform long/lat to distance
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [3]:
#define long/lat center of vienna
latitude_center = 48.210033
longitude_center = 16.363449

In [4]:
#create new column distance_to_center
for index, row in data.iterrows():
    data.loc[index, 'distance_to_center'] = haversine(row['longitude'], row['latitude'], longitude_center, latitude_center)

In [5]:
#extract year from transaction_ym
date = data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")

data["year"] = date.dt.year

In [6]:
#create new column relative_share
relative_share = data.plot_area * data.plot_share
data["relative_share"] = relative_share

In [7]:
#fill NA/NaN with mean
data = data.fillna(data.mean())

In [8]:
data["land_use"] = data["land_use"].astype(str)
data["district"] = data["district"].astype(str)
data["property_type"] = data["property_type"].astype(str)
data["contract_type"] = data["contract_type"].astype(str)
data["protection_zone"] = data["protection_zone"].astype(str)
data["building_ban"] = data["building_ban"].astype(str)

In [9]:
#split data into training and test data
data_train = data[data.year <= 2010]
data_test = data[data.year > 2010]

In [10]:
#filter outliers
data_train = data_train[data.distance_to_center < 15]
data_train = data_train[data.price_per_m2 < 2500]
data.shape

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


(44366, 21)

In [11]:
#display shape of training data
data_train.shape

(38318, 21)

In [12]:
#display shape of testing data
data_test.shape

(4316, 21)

In [13]:
#define numerical and categorical pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from future_encoders import OneHotEncoder

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('one_hot', OneHotEncoder())
])

In [14]:
#define features for pipeline A
#pip_data_a = data[["building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]
pip_data_a = data[["building_floors", "year", "distance_to_center", "relative_share"]]
pip_data_a_num = data[["year", "distance_to_center", "building_floors", "relative_share"]]
#pip_data_a_cat = data[["land_use", "district"]]

In [15]:
#define full_pipeline_a for training data_a
from future_encoders import ColumnTransformer

num_attribs = list(pip_data_a_num)
#cat_attribs = list(pip_data_a_cat)

full_pipeline_a = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
#    ('cat', cat_pipeline, cat_attribs)
])

full_pipeline_a.fit_transform(pip_data_a)

array([[ 1.72687487,  1.07481832, -0.86724244,  0.2488543 ],
       [ 1.72687487,  1.61541514, -0.05170695, -0.0859057 ],
       [ 1.72687487,  0.89049187, -0.05170695, -0.0879487 ],
       ...,
       [-1.55580778, -0.94233947,  0.        , -0.01449802],
       [-1.55580778,  0.        ,  0.        ,  0.08502523],
       [-1.55580778,  0.        ,  0.        ,  0.71640926]])

In [16]:
#define training data for pipeline A
data_train_a = data_train[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]

data_train_labels_a = data_train_a["price_per_m2"].copy()
data_train_a = data_train_a.drop("price_per_m2", axis=1)

In [17]:
#define features for pipeline B
#pip_data_b = data[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]
#pip_data_b_num = data[["building_floors", "building_year"]]
pip_data_b = data[["property_type","district","contract_type","protection_zone","building_ban", "land_use"]]
pip_data_b_cat = data[["property_type","district","contract_type","protection_zone","building_ban", "land_use"]]

In [18]:
#define full_pipeline_b for training data_b
#num_attribs = list(pip_data_b_num)
cat_attribs = list(pip_data_b_cat)

full_pipeline_b = ColumnTransformer([
#    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

full_pipeline_b.fit_transform(pip_data_b)

<44366x112 sparse matrix of type '<class 'numpy.float64'>'
	with 266196 stored elements in Compressed Sparse Row format>

In [19]:
#define training data for pipeline B
data_train_b = data_train[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]
#data_train_b_num = data_train[["building_floors", "building_year"]]
data_train_b_cat = data_train[["property_type","district","contract_type","protection_zone","building_ban", "land_use"]]

data_train_labels_b = data_train_b["price_per_m2"].copy()
data_train_b = data_train_b.drop("price_per_m2", axis=1)

In [20]:
#fit data_a and data_b
data_train_prep_a = full_pipeline_a.transform(data_train_a)
data_train_prep_b = full_pipeline_b.transform(data_train_b)

In [21]:
#declare diplay_scores function for model evaluation
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())

In [22]:
#train and cv linear regression for data_prep_a
from sklearn.linear_model import LinearRegression

lin_reg_a = LinearRegression()
lin_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(lin_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores_a)

Scores: [460.098951   416.91848152 389.01649496 343.28852796 372.60097076
 384.1674869  384.34709885 396.09503497 337.74094231 312.93745891]
Mean: 379.72114481399154
Standard Deviation 39.79992633426224


In [23]:
#train and cv linear regression for data_prep_b
lin_reg_b = LinearRegression()
lin_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(lin_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores_b)

Scores: [433.3323076  382.6479144  371.45178862 313.58262247 325.94772305
 320.78094357 329.12602227 324.19203914 333.9823381  400.68141556]
Mean: 353.5725114768766
Standard Deviation 38.766153400139984


In [24]:
#train and cv ridge regression for data_prep_a
from sklearn.linear_model import Ridge

ridge_reg_a = Ridge(alpha=0.9)
ridge_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(ridge_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores_a)

Scores: [460.09925868 416.91837789 389.01689422 343.28733804 372.59848455
 384.16706217 384.34705935 396.09441312 337.74119935 312.93884823]
Mean: 379.72089356023224
Standard Deviation 39.799850501559


In [25]:
#train and cv ridge regression for data_prep_b
ridge_reg_b = Ridge(alpha=0.9)
ridge_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(ridge_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores_b)

Scores: [433.35514227 382.62267507 371.39064147 313.5725111  325.87305122
 320.79105894 329.08164931 324.21227431 333.96210324 400.54272824]
Mean: 353.5403835184696
Standard Deviation 38.75710851512668


In [26]:
#train and cv lasso regression for data_prep_a
from sklearn.linear_model import Lasso

lasso_reg_a = Lasso(alpha=0.5)
lasso_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(lasso_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(lasso_reg_rmse_scores_a)

Scores: [460.18454837 416.92859854 389.09893512 343.14868098 372.28629198
 384.09860569 384.30259162 395.75025669 337.80263057 313.29251229]
Mean: 379.6893651849783
Standard Deviation 39.7574121697516


In [27]:
#train and cv lasso regression for data_prep_b
lasso_reg_b = Lasso(alpha=0.5)
lasso_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(lasso_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(lasso_reg_rmse_scores_b)

Scores: [437.49522444 383.71794987 374.69125965 316.42013947 327.08959708
 324.38640335 330.5325781  325.76890192 334.27892121 396.0342926 ]
Mean: 355.04152676868944
Standard Deviation 38.45619299458259


In [28]:
#train and cv decision tree regression for data_prep_a
from sklearn.tree import DecisionTreeRegressor

tree_reg_a = DecisionTreeRegressor()
tree_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(tree_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
tree_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(tree_reg_rmse_scores_a)

Scores: [522.23803705 489.61960493 490.80547119 454.19052089 465.70751164
 475.49717017 467.99111584 482.22606808 461.56874649 380.20147615]
Mean: 469.0045722441183
Standard Deviation 34.819042953880825


In [29]:
#train and cv decision tree regression for data_prep_b
tree_reg_b = DecisionTreeRegressor()
tree_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(tree_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
tree_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(tree_reg_rmse_scores_b)

Scores: [386.36116881 353.79263859 375.48874261 322.64054513 326.01802941
 324.46144679 323.56748246 316.53178796 333.71961941 403.08889004]
Mean: 346.5670351202731
Standard Deviation 29.574507348031705


In [30]:
#train and cv random forest regression for data_prep_a
from sklearn.ensemble import RandomForestRegressor

forest_reg_a = RandomForestRegressor()
forest_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(forest_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
forest_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(forest_reg_rmse_scores_a)

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Scores: [408.42464743 386.0132081  374.60220767 346.57766186 352.09155378
 375.1530762  377.70376855 383.65962681 360.42407142 290.96744863]
Mean: 365.5617270440222
Standard Deviation 30.061142050672437


In [31]:
#train and cv random forest regression for data_prep_b
forest_reg_b = RandomForestRegressor()
forest_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(forest_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
forest_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(forest_reg_rmse_scores_b)

Scores: [382.49569771 346.74329188 371.27912132 315.69588075 321.29379815
 317.49101501 319.27938775 312.51568043 326.51029217 398.78993679]
Mean: 341.2094101964559
Standard Deviation 30.14189790742385


In [32]:
#prepare test_data_a for testing
#test_data_a = data_test[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]
test_data_a = data_test[["price_per_m2", "building_floors", "year", "distance_to_center", "relative_share"]]

test_labels_a = test_data_a["price_per_m2"].copy()
test_set_a = test_data_a.drop("price_per_m2", axis=1)

test_data_prep_a = full_pipeline_a.transform(test_set_a)

#prepare test_data_b for testing
#test_data_b = data_test[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]
test_data_b = data_test[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "land_use"]]

test_labels_b = test_data_b["price_per_m2"].copy()
test_set_b = test_data_b.drop("price_per_m2", axis=1)

test_data_prep_b = full_pipeline_b.transform(test_set_b)

In [33]:
#prepare predictions on test set

#predictions from A-Models
lin_reg_pred_a = lin_reg_a.predict(test_data_prep_a)
ridge_reg_pred_a = ridge_reg_a.predict(test_data_prep_a)
lasso_reg_pred_a = lasso_reg_a.predict(test_data_prep_a)
tree_reg_pred_a = tree_reg_a.predict(test_data_prep_a)
forest_reg_pred_a = forest_reg_a.predict(test_data_prep_a)

#predictions from B-Models
lin_reg_pred_b = lin_reg_b.predict(test_data_prep_b)
ridge_reg_pred_b = ridge_reg_b.predict(test_data_prep_b)
lasso_reg_pred_b = lasso_reg_b.predict(test_data_prep_b)
tree_reg_pred_b = tree_reg_b.predict(test_data_prep_b)
forest_reg_pred_b = forest_reg_b.predict(test_data_prep_b)

In [34]:
#used metrics for experiment
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [35]:
#get metrics for models

#metrics for A-Models
lin_mse_a = np.sqrt(mean_squared_error(test_labels_a, lin_reg_pred_a))
lin_mae_a = mean_absolute_error(test_labels_a, lin_reg_pred_a)
ridge_mse_a = np.sqrt(mean_squared_error(test_labels_a, ridge_reg_pred_a))
ridge_mae_a = mean_absolute_error(test_labels_a, ridge_reg_pred_a)
lasso_mse_a = np.sqrt(mean_squared_error(test_labels_a, lasso_reg_pred_a))
lasso_mae_a = mean_absolute_error(test_labels_a, lasso_reg_pred_a)
tree_mse_a = np.sqrt(mean_squared_error(test_labels_a, tree_reg_pred_a))
tree_mae_a = mean_absolute_error(test_labels_a, tree_reg_pred_a)
forest_mse_a = np.sqrt(mean_squared_error(test_labels_a, forest_reg_pred_a))
forest_mae_a = mean_absolute_error(test_labels_a, forest_reg_pred_a)


#metrics for B-Models
lin_mse_b = np.sqrt(mean_squared_error(test_labels_b, lin_reg_pred_b))
lin_mae_b = mean_absolute_error(test_labels_b, lin_reg_pred_b)
ridge_mse_b = np.sqrt(mean_squared_error(test_labels_b, ridge_reg_pred_b))
ridge_mae_b = mean_absolute_error(test_labels_b, ridge_reg_pred_b)
lasso_mse_b = np.sqrt(mean_squared_error(test_labels_b, lasso_reg_pred_b))
lasso_mae_b = mean_absolute_error(test_labels_b, lasso_reg_pred_b)
tree_mse_b = np.sqrt(mean_squared_error(test_labels_b, tree_reg_pred_b))
tree_mae_b = mean_absolute_error(test_labels_b, tree_reg_pred_b)
forest_mse_b = np.sqrt(mean_squared_error(test_labels_b, forest_reg_pred_b))
forest_mae_b = mean_absolute_error(test_labels_b, forest_reg_pred_b)

In [36]:
df_a = pd.DataFrame(columns=['Linear/RMSE', 'Linear/MAE', 'Ridge/RMSE', 'Ridge/MAE', 'Lasso/RMSE', 'Lasso/MAE', 'Tree/RMSE', 'Tree/MAE', 'Forest/RMSE', 'Forest/MAE'])
df_a['Linear/RMSE'] = [lin_mse_a]
df_a['Linear/MAE'] = [lin_mae_a]
df_a['Ridge/RMSE'] = [ridge_mse_a]
df_a['Ridge/MAE'] = [ridge_mae_a]
df_a['Lasso/RMSE'] = [lasso_mse_a]
df_a['Lasso/MAE'] = [lasso_mae_a]
df_a['Tree/RMSE'] = [tree_mse_a]
df_a['Tree/MAE'] = [tree_mae_a]
df_a['Forest/RMSE'] = [forest_mse_a]
df_a['Forest/MAE'] = [forest_mae_a]

df_a

,Linear/RMSE,Linear/MAE,Ridge/RMSE,Ridge/MAE,Lasso/RMSE,Lasso/MAE,Tree/RMSE,Tree/MAE,Forest/RMSE,Forest/MAE
0,2627.123471,968.478548,2627.128257,968.480335,2627.892243,968.777952,2576.102179,910.522472,2542.940352,859.173018


In [37]:
df_b = pd.DataFrame(columns=['Linear/RMSE', 'Linear/MAE', 'Ridge/RMSE', 'Ridge/MAE', 'Lasso/RMSE', 'Lasso/MAE', 'Tree/RMSE', 'Tree/MAE', 'Forest/RMSE', 'Forest/MAE'])
df_b['Linear/RMSE'] = [lin_mse_b]
df_b['Linear/MAE'] = [lin_mae_b]
df_b['Ridge/RMSE'] = [ridge_mse_b]
df_b['Ridge/MAE'] = [ridge_mae_b]
df_b['Lasso/RMSE'] = [lasso_mse_b]
df_b['Lasso/MAE'] = [lasso_mae_b]
df_b['Tree/RMSE'] = [tree_mse_b]
df_b['Tree/MAE'] = [tree_mae_b]
df_b['Forest/RMSE'] = [forest_mse_b]
df_b['Forest/MAE'] = [forest_mae_b]

df_b

,Linear/RMSE,Linear/MAE,Ridge/RMSE,Ridge/MAE,Lasso/RMSE,Lasso/MAE,Tree/RMSE,Tree/MAE,Forest/RMSE,Forest/MAE
0,2636.074095,929.82819,2636.575413,929.75144,2661.902579,936.862348,2587.737949,867.419227,2592.944415,868.732417


In [38]:
sorted(zip(tree_reg_b.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.3122143775307896, 'land_use'),
 (0.02001007124218196, 'contract_type'),
 (0.005757497903871896, 'building_floors'),
 (0.004127580434368415, 'distance_to_center'),
 (0.0035894106604006572, 'building_ban'),
 (0.0034408202444856277, 'property_type'),
 (0.0031016799858435327, 'protection_zone'),
 (0.0025208032352823664, 'year'),
 (0.0018874211130663289, 'relative_share'),
 (5.558522331977779e-06, 'district')]

In [39]:
sorted(zip(tree_reg_a.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.443651961620323, 'distance_to_center'),
 (0.2685396685638521, 'relative_share'),
 (0.1702794682117064, 'year'),
 (0.11752890160411865, 'building_floors')]

In [40]:
sorted(zip(forest_reg_b.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.3038043603075613, 'land_use'),
 (0.01135244107752148, 'contract_type'),
 (0.006958199844920035, 'building_floors'),
 (0.00625017049623265, 'building_ban'),
 (0.00556929373164231, 'protection_zone'),
 (0.005491783860257404, 'property_type'),
 (0.004729613635281249, 'distance_to_center'),
 (0.004274513134595376, 'year'),
 (0.0014282913149084376, 'relative_share'),
 (4.435313207412804e-06, 'district')]

In [41]:
sorted(zip(forest_reg_a.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.4429683744844026, 'distance_to_center'),
 (0.2698534319667941, 'relative_share'),
 (0.1725522426557861, 'year'),
 (0.11462595089301728, 'building_floors')]

In [42]:
pd.concat([df_a, df_b])

,Linear/RMSE,Linear/MAE,Ridge/RMSE,Ridge/MAE,Lasso/RMSE,Lasso/MAE,Tree/RMSE,Tree/MAE,Forest/RMSE,Forest/MAE
0,2627.123471,968.478548,2627.128257,968.480335,2627.892243,968.777952,2576.102179,910.522472,2542.940352,859.173018
0,2636.074095,929.828190,2636.575413,929.751440,2661.902579,936.862348,2587.737949,867.419227,2592.944415,868.732417
